# Ограниченная машина Больцмана (Restricted Boltzmann machine)

## Введение

<b> Ограниченная машина Больцмана (RBM) </b> — это генеративная стохастическая искусственная нейронная сеть, которая определяет распределение вероятности на наборе входных данных.

Как следует из их названия, RBM являются вариантом машин Больцмана с ограничением, что их нейроны должны образовывать двудольный граф: пара узлов, сформированная из нейронов двух групп (обычно называемых «видимыми» и «скрытыми» соответственно), имеет симметричное соединение между собой, и нет никаких связей между узлами внутри группы. Это ограничение позволяет использовать более эффективные алгоритмы обучения, чем те, которые доступны для общего класса машин Больцмана, в частности, алгоритм контрастной дивергенции на основе градиента. Кроме того, особенностью ограниченных машин Больцмана является возможность прохождения обучение без учителя.

## Обозначения и вывод формул

Введем следующие обозначения:
* Состояния нейронов видимого слоя
$$v_1, v_2, ..., v_{N_v}; v_i \in \{0, 1\}, i = \overline{1, N_v}, v \in R^{N_v}$$
* Состояния нейронов скрытого слоя
$$h_1, h_2, ..., h_{N_h}; h_i \in \{0, 1\}, i = \overline{1, N_h}, h \in R^{N_h}$$
* Матрица связей между нейронами видимого и скрытого слоя
$$W \in R^{N_v \times N_h}$$
* Веc между i-ым нейроном видимого слоя и j-ым нейроном скрытого слоя
$$w_{ij}, i = \overline{1, N_v}, j = \overline{1, N_h}$$
* Смещение видимого нейрона
$$a_i, i = \overline{1, N_v}$$
* Смещение скрытого нейрона
$$b_j, j = \overline{1, N_h}$$

![RBM1](https://raw.githubusercontent.com/Frixinglife/RBM/adb5aa904ca892d3be8f05867935dc8075efc052/images/RBM1.svg)

### Энергия и вероятности

* Введем понятие энергии для ограниченной машины Больцмана:
$$E(v, h) = -\sum_{i=1}^{N_v}a_iv_i-\sum_{j=1}^{N_h}b_jh_j-\sum_{i=1}^{N_v}\sum_{j=1}^{N_h}w_{ij}v_ih_j=-a^\top v-b^\top h - v^\top W h$$

* Вероятность всевозможных пар v и h:
$$p(v, h) = \frac{1}{Z}e^{-E(v, h)},$$
где Z — это статсумма следующего вида (суммирование по всевозможным векторам v и h):
$$Z = \sum_v\sum_h e^{-E(v, h)}$$

* Полная вероятность вектора v (суммирование по всевозможным векторам h):
$$p(v) = \frac{1}{Z}\sum_h e^{-E(v, h)}$$

### Вычисление условных вероятностей

* Вероятность того, что при данном v скрытое состояние h<sub>j</sub> = 1:
$$p(h_j = 1|v) = \frac{e^{-E(v,h_j = 1)}}{e^{-E(v,h_j = 1)} + e^{-E(v,h_j = 0)}}=\frac{1}{1 + e^{E(v,h_j = 1)-E(v,h_j = 0)}}=\frac{1}{1 + e^{-b_j-\sum_{i=1}^{N_v}w_{ij}v_i}}=\sigma(b_j+\sum_{i=1}^{N_v}w_{ij}v_i),$$
где
$$\sigma(z)=\frac{1}{1 + e^{-z}}$$
сигмоидальная функция

* Вероятность того, что при данном h видимое состояние v<sub>i</sub> = 1:
$$p(v_i = 1|h) = \frac{e^{-E(v_i = 1,h)}}{e^{-E(v_i = 1,h)} + e^{-E(v_i = 0,h)}}=\frac{1}{1 + e^{E(v_i = 1,h)-E(v_i = 0,h)}}=\frac{1}{1 + e^{-a_i-\sum_{j=1}^{N_h}w_{ij}h_j}}=\sigma(a_i+\sum_{j=1}^{N_h}w_{ij}h_j)$$

* Поскольку при фиксированном v скрытые состояния h<sub>j</sub> независимы друг от друга (как и при фиксированном h видимые состояния v<sub>i</sub>), получаем:
$$p(h|v)=\prod_{j=1}^{N_h}p(h_j|v)$$
$$p(v|h)=\prod_{i=1}^{N_v}p(h|v_i)$$

### Цель обучения

Мы можем наблюдать состояния видимых нейронов, но не можем напрямую увидеть состояния скрытых. Несмотря на это, опираясь на наблюдаемые состояния, можно сделать вероятностный вывод относительно скрытых состояний. После обучения данной модели мы сможем делать вероятностные выводы (при помощи теоремы Байеса) о том, какими являются видимые нейроны, опираясь на скрытые. Таким образом, приходим к следующей <b>цели</b>:

> Необходимо обучить модель таким образом (настроить её параметры), чтобы полученный в результате работы RBM вектор состояния был близок к первоначальному состоянию, из которого он был восстановлен.

Данную цель можно достигнуть путем максимизации вероятности p(v). Кроме того, вместо максимизации вероятности p(v), можно максимизировать натуральный логарифм от нее (это удобнее для получения формул), поскольку он является строго возрастающей функцией.

### Максимизация вероятности p(v)

Найдем производную от натурального логарифма p(v) по весу w<sub>ij</sub>:
$$\frac{\partial \log(p(v))}{\partial w_{ij}} = \frac{1}{p(v)} \frac{\partial p(v)}{\partial w_{ij}} = \frac{1}{p(v)}\frac{\partial}{\partial w_{ij}}(\frac{\sum_h e^{-E(v, h)}}{Z})= \frac{1}{p(v)}\frac{\frac{\partial}{\partial w_{ij}}(\sum_h e^{-E(v, h)})Z - \sum_h e^{-E(v, h)}\frac{\partial Z}{\partial w_{ij}}}{Z^2},$$

где
$$p(v) = \frac{1}{Z}\sum_h e^{-E(v, h)}$$
$$Z = \sum_v\sum_h e^{-E(v, h)}$$
$$E(v, h) = -\sum_{i=1}^{N_v}a_iv_i-\sum_{j=1}^{N_h}b_jh_j-\sum_{i=1}^{N_v}\sum_{j=1}^{N_h}w_{ij}v_ih_j$$

Вычислим производные:
$$\frac{\partial}{\partial w_{ij}}(\sum_h e^{-E(v, h)})=\sum_h v_ih_je^{-E(v, h)}$$
$$\frac{\partial Z}{\partial w_{ij}}=\frac{\partial}{\partial w_{ij}}(\sum_v\sum_h e^{-E(v, h)})=\sum_v\sum_h v_ih_je^{-E(v, h)}$$

После подстановки получаем:
$$\frac{\partial \log(p(v))}{\partial w_{ij}}=\frac{1}{p(v)}\frac{\frac{\partial}{\partial w_{ij}}(\sum_h e^{-E(v, h)})Z - \sum_h e^{-E(v, h)}\frac{\partial Z}{\partial w_{ij}}}{Z^2}=\frac{Z}{\sum_h e^{-E(v, h)}}\frac{Z(\sum_h v_ih_je^{-E(v, h)}) - (\sum_h e^{-E(v, h)})(\sum_v\sum_h v_ih_je^{-E(v, h)})}{Z^2}=\frac{\sum_h v_ih_je^{-E(v, h)}}{\sum_h e^{-E(v, h)}}-\frac{\sum_h e^{-E(v, h)}}{Z} \frac{\sum_v\sum_h v_ih_je^{-E(v, h)}}{\sum_h e^{-E(v, h)}}=\frac{\sum_h v_ih_je^{-E(v, h)}}{\sum_h e^{-E(v, h)}}-p(v) \frac{\sum_v\sum_h v_ih_je^{-E(v, h)}}{\sum_h e^{-E(v, h)}},$$

Домножим числитель и знаменатель каждого слагаемого на Z: